<a href="https://colab.research.google.com/github/maxwellowusu/youthmappers_activities/blob/main/HOT_Routine_trackers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HOT Routine trackers

This codes analyses YouthMappers project activities in the HOT Tasking manager. 


Created by: Maxwell Owusu

Spring 2022

The George Washington University

In [12]:
# import libraries

import requests
import json
import pandas as pd
import numpy as np

from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Setup HOT-TM to download YouthMappers activities 
apiKey='Token asdgshswrthdhsdfghbtdhasertsb'
hasNext=True
results = []
page = 1

while hasNext:
    payload = {'organisationId': 5, 'page': page, 'projectStatuses': 'PUBLISHED,ARCHIVED' }
    headers = {'Authorization': 'apiKey'}
    r = requests.get("https://tasking-manager-tm4-production-api.hotosm.org/api/v2/projects/", params=payload, headers=headers)
    response = r.json()
    response['pagination']['page']
    results.append(response)
    if response['pagination']['hasNext']:
        page = response['pagination']['nextNum']
    else:
        hasNext=False

In [3]:
# Get the list of YM project IDs

# rows list initialization
projectsID = []
status = []
percentMapped = []
percentValidated = []
organisationName = []
lastUpdated = []
# appending rows
for data in results:
    data_row = data['results']
    # print (data_row)

    for row in data_row:
        row= row['projectId']
        projectsID.append(row)

    for row in data_row:
        row= row['status']
        status.append(row)

    for row in data_row:
        row= row['percentMapped']
        percentMapped.append(row)

    for row in data_row:
        row= row['percentValidated']
        percentValidated.append(row)

    for row in data_row:
        row= row['organisationName']
        organisationName.append(row)

    for row in data_row:
        row= row['lastUpdated']
        lastUpdated.append(row)

# create a list to selected features

df_list = {"projectsID": projectsID, "status": status, "percentMapped": percentMapped, "percentValidated": percentValidated, "organisationName": organisationName, "lastUpdated": lastUpdated}
# convert list to dataframe
df = pd.DataFrame(df_list)
print(df.shape)
df.head(10)

(120, 6)


,projectsID,status,percentMapped,percentValidated,organisationName,lastUpdated
0,7932,ARCHIVED,100,99,YouthMappers,2020-02-23T18:55:08.561000Z
1,1193,ARCHIVED,100,86,YouthMappers,2021-09-27T20:55:55.970500Z
2,1957,ARCHIVED,65,37,YouthMappers,2022-02-17T17:59:29.307755Z
3,2727,ARCHIVED,85,17,YouthMappers,2021-05-05T13:11:47.369231Z
4,2728,ARCHIVED,92,57,YouthMappers,2021-05-05T13:12:31.875612Z
5,2742,ARCHIVED,100,100,YouthMappers,2021-05-05T13:14:28.369353Z
6,3713,ARCHIVED,91,41,YouthMappers,2021-08-12T22:06:43.638639Z
7,3717,ARCHIVED,100,33,YouthMappers,2021-08-12T22:05:56.394977Z
8,3859,ARCHIVED,92,57,YouthMappers,2021-04-18T17:32:10.147682Z
9,4147,ARCHIVED,44,0,YouthMappers,2018-05-16T18:36:32.619567Z


In [4]:
projectIds = []
YM_contributions_results = []
project_ids = df["projectsID"].values
hot_base_url = 'https://tasking-manager-tm4-production-api.hotosm.org'

# Retrieve relevant project meta data information via HOT Tasking Manager API
# https://tasking-manager-tm4-production-api.hotosm.org/api/v2/projects/1234/

for project_id in project_ids:
  project_api_endpoint = f'{hot_base_url}/api/v2/projects/{project_id}/'
  project_info = requests.get(project_api_endpoint).json()
  YM_contributions_results.append({project_id : project_info})
  # projectIds.append({"projectid": project_id})

print(len(YM_contributions_results))

120


In [5]:
# rows list initialization
name = []
username = []
mappingLevel = []
mapped = []  
validated =[]
total = []
projectIds = []
created = []
# appending rows

for data in YM_contributions_results:
  # print (data)
  for p_id, p_info in data.items():
    projectIds.append(p_id)
    data_row = p_info['created']
    created.append(data_row)
    # print (data_row)



print(len(projectIds))
print(len(created))

df_list = {"projectsID": projectIds, "created": created}
# convert list to dataframe
df1 = pd.DataFrame(df_list)
print(df1.shape)
# df1.head(10)

120
120
(120, 2)


In [6]:
# Merge data
df_full  = pd.concat([df,df1], axis=1)


In [10]:
df_full['date_created'] = df_full['created'].apply(pd.to_datetime)
df_full['date'] = pd.to_datetime(df_full['created']).dt.date
df_full.sort_values(by='date', ascending=False, inplace=True)
df_full.drop(['date_created', 'created'], axis=1, inplace=True)
df_full.head(15)


,projectsID,status,percentMapped,percentValidated,organisationName,lastUpdated,projectsID,date
119,13650,PUBLISHED,51,15,YouthMappers,2022-11-28T10:18:54.874459Z,13650,2022-10-24
117,13582,PUBLISHED,66,1,YouthMappers,2022-11-16T12:25:47.270245Z,13582,2022-10-01
118,13583,PUBLISHED,3,0,YouthMappers,2022-11-09T17:25:57.230844Z,13583,2022-10-01
116,13455,PUBLISHED,1,0,YouthMappers,2022-09-19T18:27:53.497706Z,13455,2022-09-03
115,13209,PUBLISHED,100,49,YouthMappers,2022-11-29T13:28:52.919025Z,13209,2022-08-09
114,13169,PUBLISHED,100,70,YouthMappers,2022-09-18T06:11:28.032387Z,13169,2022-08-09
113,13046,PUBLISHED,71,0,YouthMappers,2022-09-03T19:09:00.212021Z,13046,2022-07-23
112,13015,PUBLISHED,70,1,YouthMappers,2022-12-06T08:05:13.323469Z,13015,2022-07-19
75,12997,PUBLISHED,14,0,YouthMappers,2022-11-15T08:50:01.819526Z,12997,2022-07-13
111,12975,PUBLISHED,100,36,YouthMappers,2022-09-08T09:32:40.405393Z,12975,2022-07-11


In [16]:
# Make directory
import os

OUTPUT_PATH = "/content/drive/MyDrive/Colab Notebooks/youthmappers/output"
directory = os.path.dirname(OUTPUT_PATH)
if not os.path.exists(directory):
    os.makedirs(directory)

In [19]:
# Save results to csv
df_full.to_csv(f'{OUTPUT_PATH}/results.csv')